# Create Data Structure

Using the data model json, create a structure as file geodatabase

#### Import statements

In [1]:
# import the arcpy libary
import arcpy
arcpy.env.overwriteOutput = True

# import other modules
import json
import copy

# set default spatial reference
sr = arcpy.SpatialReference(r'C:\Users\rjc.EAGLE\AppData\Local\ESRI\ArcGISPro\Favorites\NZGD 2000 New Zealand Transverse Mercator.prj')

#### Set paths and load in Data Model

In [4]:
basePath = r'C:\Users\rjc.EAGLE\Documents\GitHub\AgricultureDataModel'

data = {}
data.update(json.load(open('{0}\\json\\{1}.json'.format(basePath, 'datamodel'))))
data.update(json.load(open('{0}\\json\\{1}.json'.format(basePath, 'tables'))))
data.update(json.load(open('{0}\\json\\{1}.json'.format(basePath, 'domains'))))
data

{'services': [{'name': 'administrative_information',
   'alias': 'Administrative Information',
   'tables': ['activities', 'additional_details', 'measurements', 'staff'],
   'layers': [{'name': 'access_points',
     'alias': 'Access Points',
     'type': 'POINT',
     'children': ['activities'],
     'fields': [{'name': 'name',
       'alias': 'Address',
       'type': 'TEXT',
       'length': 255,
       'domain': None},
      {'name': 'alias',
       'alias': 'Alias',
       'type': 'TEXT',
       'length': 255,
       'domain': None},
      {'name': 'unique_number',
       'alias': 'Unique Number',
       'type': 'SHORT',
       'length': None,
       'domain': None},
      {'name': 'status',
       'alias': 'Status',
       'type': 'TEXT',
       'length': 255,
       'domain': 'status'},
      {'name': 'start_date',
       'alias': 'Start Date',
       'type': 'DATE',
       'length': None,
       'domain': None},
      {'name': 'end_date',
       'alias': 'End Date',
       'type

#### Define functions

In [3]:
def createItems(workspace, item):   
    if item['type'] == 'TABLE':        
        result = arcpy.management.CreateTable(workspace, item['name'], out_alias=item['alias'])        
    else:
        result = arcpy.management.CreateFeatureclass(workspace, item['name'], item['type'], spatial_reference=sr, out_alias=item['alias'])
    itemFields = item['fields']
    if itemFields != None:
        fields = itemFields
    else:
        table = [x for x in data['tables'] if item['master'] == x['name']][0]
        fields = table['fields']
    for field in fields:    
        arcpy.management.AddField(result, field['name'], field['type'], field_length=field['length'], field_alias=field['alias'], field_domain=field['domain'])
        if field['name'] == 'status':
            arcpy.management.AssignDefaultToField(result, field['name'], "Current")
        
    arcpy.management.AddGlobalIDs(result)
    print("    + {0}".format(item['name']))

def createRelationships(workspace, item):  
    origin_table = '{0}\\{1}'.format(workspace, item['name'])     
    parent_id = 'parent_{0}_id'.format(item['name'])
    parent_alias = 'Parent {0} ID'.format(item['name'].replace('_', ' '))
    children = item['children']
    for child in children:
        child_path = '{0}\\{1}'.format(workspace, child)       
        relationship_class = '{0}_{1}'.format(item['name'], child)
        arcpy.management.AddField(child_path, parent_id, 'GUID') 
        arcpy.management.CreateRelationshipClass(origin_table, child_path, relationship_class, 'Simple', child, item['name'], None, 'ONE_TO_MANY', None, 'GlobalID', parent_id)  
    print("    + {0}".format(item['name']))
        
def enableEditorTracking(workspace, item): 
    in_table = '{0}\\{1}'.format(workspace, item['name'])
    arcpy.management.AddField(in_table, field_name='creator', field_type='TEXT', field_alias='Creator')
    arcpy.management.AddField(in_table, field_name='creation_date', field_type='DATE', field_alias='Creation Date')
    arcpy.management.AddField(in_table, field_name='last_editor', field_type='TEXT', field_alias='Last Editor')
    arcpy.management.AddField(in_table, field_name='last_edit_date', field_type='DATE', field_alias='Last Edit Date')
    arcpy.management.EnableEditorTracking(in_table, 'creator', 'creation_date', 'last_editor', 'last_edit_date') 
    arcpy.management.EnableAttachments(in_table)
    print("    + {0}".format(item['name']))

#### Create data structure

In [5]:
outputDataPath = '{0}\\output\\data'.format(basePath)
arcpy.env.workspace = outputDataPath
existingWorkspaces = arcpy.ListWorkspaces("*", "FileGDB")

for service in data['services']:
    
    print(service['name'])
    
    # creat the workspace, this will overwrite the existing workspaces
    workspace = arcpy.CreateFileGDB_management(outputDataPath, "{0}.gdb".format(service['name'])).getOutput(0)
    arcpy.env.workspace = workspace
    
    print("  - creating domains")
    #existing_domains = [x.name for x in arcpy.da.ListDomains(workspace)]  
    domains = [x for x in data['domains'] if service['name'] in x['workspace'] or '*' == x['workspace'][0]]
    for domain in domains:
        arcpy.management.CreateDomain(workspace, domain['name'], domain['description'], domain['field_type'], domain['domain_type'])
        for code in domain['codes']:
            arcpy.AddCodedValueToDomain_management(workspace, domain['name'], code, domain['codes'][code])
    print("    + completed")
    
    # create the tables
    print("  - creating tables")
    tables = [x for x in data['tables'] if x['name'] in service['tables']]
    for table in tables:        
        createItems(workspace, table)
           
    # create items
    print("  - creating items")
    layers = service['layers']
    for item in layers:        
        createItems(workspace, item) 
       
    # create relationships
    print("  - creating relationships")
    for item in layers:        
        createRelationships(workspace, item)  
        
    # enable editor tracking
    print("  - enabling editor tracking")
    for item in layers:        
        enableEditorTracking(workspace, item)  
        
    print(' ')


administrative_information
  - creating domains
    + completed
  - creating tables
    + activities
    + measurements
    + additional_details
    + staff
  - creating items
    + access_points
    + blocks
    + site
    + holding
  - creating relationships
    + access_points
    + blocks
    + site
    + holding
  - enabling editor tracking
    + access_points
    + blocks
    + site
    + holding
 
structures
  - creating domains
    + completed
  - creating tables
    + activities
    + hazards
    + measurements
    + additional_details
  - creating items
    + production_structures
    + storage_structures
    + service_structures
    + management_structures
    + waste_structures
  - creating relationships
    + production_structures
    + storage_structures
    + service_structures
    + management_structures
    + waste_structures
  - enabling editor tracking
    + production_structures
    + storage_structures
    + service_structures
    + management_structures
    + wast